In [ ]:
import numpy as np
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

: 

In [2]:
df = pd.read_csv(r'.\booklist.csv', index_col=False)

col_names=['bookId','title','series','author','rating','description','lang','isbn','genres','characters','bookForm','edition','pages','publisher','publishDate','firstPublished','awards','numRating','ratingsByStars','likedPercent','setting','coverImg','bbeScore','bbeVotes','price']
wanted_cols = ['bookId','title','series','author','rating','description','genres','characters','setting','coverImg']

#TODO: filter out our wanted cols, create our index and save it to a file(?)
filtered_df = df[wanted_cols].head(5)

print(df['genres'].dtypes)
print(df['characters'].dtypes)
print(df['setting'].dtypes)

object
object
object


In [3]:
# Characters, author and genre
df['genres'] = df['genres'].apply(literal_eval)
df['characters'] = df['characters'].apply(literal_eval)
df['setting'] = df['setting'].apply(literal_eval)

df['author'] = df['author'].apply(lambda x : x.split(", "))
print(df['author'].dtypes)

df['soup'] = df['author'] + df['genres'] + df['characters'] + df['setting'] 

df['soup'] = df['soup'].apply(lambda x: ' '.join(x))
print(df['soup'].head(10))
print(df['soup'].dtypes)
print(df['soup'].iloc[0])

object
0    Suzanne Collins Young Adult Fiction Dystopia F...
1    J.K. Rowling Mary GrandPré (Illustrator) Fanta...
2    Harper Lee Classics Fiction Historical Fiction...
3    Jane Austen Anna Quindlen (Introduction) Class...
4    Stephenie Meyer Young Adult Fantasy Romance Va...
5    Markus Zusak (Goodreads Author) Historical Fic...
6    George Orwell Russell Baker (Preface) C.M. Woo...
7    C.S. Lewis Pauline Baynes (Illustrator) Fantas...
8    J.R.R. Tolkien Fantasy Fiction Classics Advent...
9    Margaret Mitchell Classics Historical Fiction ...
Name: soup, dtype: object
object
Suzanne Collins Young Adult Fiction Dystopia Fantasy Science Fiction Romance Adventure Teen Post Apocalyptic Action Katniss Everdeen Peeta Mellark Cato (Hunger Games) Primrose Everdeen Gale Hawthorne Effie Trinket Haymitch Abernathy Cinna President Coriolanus Snow Rue Flavius Lavinia (Hunger Games) Marvel Glimmer Clove Foxface Thresh Greasy Sae Madge Undersee Caesar Flickerman Claudius Templesmith Octavia (

In [4]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df['soup'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [5]:
indices = pd.Series([i for i in range(len(df))] ,index=df['title'])
print(indices.head(10))

title
The Hunger Games                                                         0
Harry Potter and the Order of the Phoenix                                1
To Kill a Mockingbird                                                    2
Pride and Prejudice                                                      3
Twilight                                                                 4
The Book Thief                                                           5
Animal Farm                                                              6
The Chronicles of Narnia                                                 7
J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings    8
Gone with the Wind                                                       9
dtype: int64


In [6]:
def get_recommendations(title, no_of_recommendation):
    idx = indices[title]
    df['similarity score'] = np.array(cosine_sim[idx])
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:no_of_recommendation+1]
    print(len(sim_scores))
    book_indices = [i[0] for i in sim_scores]
    return df[["title", "similarity score"]].iloc[book_indices]

In [9]:
get_recommendations("Harry Potter and the Order of the Phoenix", 10)

10


,title,similarity score
71,Harry Potter and the Deathly Hallows,0.797096
93,Harry Potter and the Prisoner of Azkaban,0.795446
103,Harry Potter and the Goblet of Fire,0.779804
105,Harry Potter and the Half-Blood Prince,0.706655
126,Harry Potter and the Chamber of Secrets,0.705789
32,Harry Potter and the Sorcerer's Stone,0.675333
2580,Harry Potter and the Methods of Rationality,0.409137
7418,Harry Potter: Film Wizardry,0.380216
5203,James Potter and the Hall of Elders' Crossing,0.371447
1332,Harry Potter and the Cursed Child: Parts One a...,0.334309
